In [ ]:
from github import Github
from collections import Counter, defaultdict

GITHUB_TOKEN = "my token"
USERNAME = "johnatan-si"

g = Github(GITHUB_TOKEN)
user = g.get_user(USERNAME)

print(f"\n📛 Nome: {user.name}")
print(f"🔐 Login: {user.login}")
print(f"🖼️ Foto de perfil: {user.avatar_url}\n")

fotoPerfil = user.avatar_url
languageCommitCounter = Counter()
reposLanguage = defaultdict(list)

for repo in user.get_repos():
    try:
        commits = repo.get_commits()
        commit_count = commits.totalCount
        langs = repo.get_languages()

        for lang in langs:
            languageCommitCounter[lang] += commit_count
            reposLanguage[lang].append(f"{repo.name};{commit_count}")

    except Exception as e:
        print(f"⚠️ Erro ao acessar {repo.name}: {e}")

top_5_languages = languageCommitCounter.most_common(5)
top5ReposLanguage = {lang: reposLanguage[lang] for lang, _ in top_5_languages}

print("\n📊 Top 5 linguagens mais ativas (por número de commits):")
for lang, total_commits in top_5_languages:
    print(f"➡️ {lang}: {total_commits} commits")

print("\n📂 Repositórios por linguagem (Top 5):")
for lang, repos in top5ReposLanguage.items():
    print(f"\n🗂️ {lang}:")
    for repo_info in repos:
        print(f"   - {repo_info}")



📛 Nome: Johnatan 
🔐 Login: johnatan-si
🖼️ Foto de perfil: https://avatars.githubusercontent.com/u/8726738?v=4

⚠️ Erro ao acessar SourceLearning-: Git Repository is empty.: 409 {"message": "Git Repository is empty.", "documentation_url": "https://docs.github.com/rest/commits/commits#list-commits", "status": "409"}

📊 Top 5 linguagens mais ativas (por número de commits):
➡️ Java: 246 commits
➡️ Dockerfile: 111 commits
➡️ Python: 81 commits
➡️ Makefile: 59 commits
➡️ AspectJ: 58 commits

📂 Repositórios por linguagem (Top 5):

🗂️ Java:
   - astar0;2
   - AulasPUCMG;60
   - demo-ci;56
   - DownloadGit;2
   - extractgit;1
   - GitBlame;8
   - JExpert;1
   - JExpert2.0;1
   - jreuse;12
   - JSERD2020;43
   - LineCounter;2
   - ufmg.reuso.software;58

🗂️ Dockerfile:
   - F03_API;62
   - PUC-GCES-Tutoriais;49

🗂️ Python:
   - aula-ci;4
   - F03_API;62
   - github_recommend;3
   - LinesOfCodeCounter;5
   - PUC-GCES-JS;4
   - PUC-GCES-PY;3

🗂️ Makefile:
   - demo-ci;56
   - github_recommend;3



In [25]:
print(languageCommitCounter)
print(top5ReposLanguage)
print(fotoPerfil)


Counter({'Java': 246, 'Dockerfile': 111, 'Python': 81, 'Makefile': 59, 'AspectJ': 58, 'Shell': 52, 'Jupyter Notebook': 7, 'C': 6, 'JavaScript': 4, 'HTML': 4, 'CSS': 4})
{'Java': ['astar0;2', 'AulasPUCMG;60', 'demo-ci;56', 'DownloadGit;2', 'extractgit;1', 'GitBlame;8', 'JExpert;1', 'JExpert2.0;1', 'jreuse;12', 'JSERD2020;43', 'LineCounter;2', 'ufmg.reuso.software;58'], 'Dockerfile': ['F03_API;62', 'PUC-GCES-Tutoriais;49'], 'Python': ['aula-ci;4', 'F03_API;62', 'github_recommend;3', 'LinesOfCodeCounter;5', 'PUC-GCES-JS;4', 'PUC-GCES-PY;3'], 'Makefile': ['demo-ci;56', 'github_recommend;3'], 'AspectJ': ['ufmg.reuso.software;58']}
https://avatars.githubusercontent.com/u/8726738?v=4


In [ ]:
import matplotlib.pyplot as plt
import os

output_dir = "graficos_linguagens"
os.makedirs(output_dir, exist_ok=True)

print("\n📊 Gerando gráficos por linguagem...")

for lang, repos in top5ReposLanguage.items():
    nomes_repos = []
    commits = []

    for entry in repos:
        nome, commit_str = entry.split(";")
        nomes_repos.append(nome)
        commits.append(int(commit_str))

    n_repos = len(repos)
    largura_barra = 0.8 if n_repos > 1 else 0.4 

    plt.figure(figsize=(10, 6))

    plt.bar(nomes_repos, commits, color='skyblue', width=largura_barra)

    if n_repos == 1:
        plt.xlim(-0.5, 0.5) 
    plt.title(f"Commits por repositório - {lang}")
    plt.xlabel("Repositórios")
    plt.ylabel("Commits")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    filepath = os.path.join(output_dir, f"{lang}_commits.png")
    plt.savefig(filepath)
    plt.close()

    print(f"✅ Gráfico salvo: {filepath}")



📊 Gerando gráficos por linguagem...
✅ Gráfico salvo: graficos_linguagens/Java_commits.png
✅ Gráfico salvo: graficos_linguagens/Dockerfile_commits.png
✅ Gráfico salvo: graficos_linguagens/Python_commits.png
✅ Gráfico salvo: graficos_linguagens/Makefile_commits.png
✅ Gráfico salvo: graficos_linguagens/AspectJ_commits.png
